# HKN Officer Requirement Tracking

In [1]:
import os

import pandas as pd
import requests

# Importing Data

In [2]:
responses_req = requests.get("https://docs.google.com/spreadsheets/d/e/2PACX-1vTqjtYipKDHEQwx5OIho6pC_WOwKyBHMtYIqm4my9PvGrTZlHMnoYq-F68RxFhb2Hjt39HdIHB6QfpV/pub?gid=719878135&single=true&output=csv")
events_req = requests.get("https://docs.google.com/spreadsheets/d/e/2PACX-1vTqjtYipKDHEQwx5OIho6pC_WOwKyBHMtYIqm4my9PvGrTZlHMnoYq-F68RxFhb2Hjt39HdIHB6QfpV/pub?gid=930946685&single=true&output=csv")

with open(os.path.join(os.getcwd(), "responses.csv"), 'wb') as f:
    f.write(responses_req.content)

with open(os.path.join(os.getcwd(), "events.csv"), 'wb') as f:
    f.write(events_req.content)

responses = pd.read_csv(os.path.join(os.getcwd(), "responses.csv"))
events = pd.read_csv(os.path.join(os.getcwd(), "events.csv"))

# Pre Processing

In [3]:
responses["HKN Handle"] = responses["HKN Handle"].str.strip().str.lower()
responses["Secret Word"] = responses["Secret Word"].str.strip().str.lower()
events["Secret Word"] = events["Secret Word"].str.strip().str.lower()

In [4]:
def count_attendance(responses: pd.DataFrame, events: pd.DataFrame, column: str) -> pd.DataFrame:
    filtered_events = events[events["Activity Type"] == column]
    filtered_responses = responses[["HKN Handle", "Week", "Secret Word"]][responses["Activity Type"] == column]

    df_dict = dict(tuple(filtered_responses.groupby(["HKN Handle"])))

    return pd.DataFrame(
        [(k, v.merge(filtered_events, how="inner", on=["Week", "Secret Word"]).shape[0]) for k, v in df_dict.items()],
        columns=["HKN Handle", f"{column}s Attended"],
    )

# HM Attendance

In [5]:
hm_attendance = count_attendance(responses, events, "HM")
hm_attendance

,HKN Handle,HMs Attended
0,aaronguo,3
1,adrianwei,3
2,aeroanish,2
3,albertguo,3
4,alexanderfung,2
...,...,...
64,varunbharadwaj,3
65,vincentwang,1
66,vivianwuc,1
67,wgiorza,3


# Cookie Run Attendance

In [6]:
cr_attendance = count_attendance(responses, events, "Cookie Run")
cr_attendance

,HKN Handle,Cookie Runs Attended
0,aaronguo,2
1,albertguo,1
2,anson.tiong,2
3,arengarajan,2
4,ayushi,1
5,baoer,1
6,bhatsavit,1
7,bryanngo,2
8,dvaish,1
9,ibork,1


# Teaching Sessions Attendance

In [7]:
ts_attendance = count_attendance(responses, events, "Teaching Session")
ts_attendance

,HKN Handle,Teaching Sessions Attended
0,anson.tiong,1
1,franzkieviet,1
2,jeffdeng,1


# Final Table

In [8]:
attendance = hm_attendance.merge(cr_attendance, how="outer", on="HKN Handle").merge(ts_attendance, how="outer", on="HKN Handle").fillna(0)
attendance.iloc[:, 1:] = attendance.iloc[:, 1:].astype(int)
attendance

,HKN Handle,HMs Attended,Cookie Runs Attended,Teaching Sessions Attended
0,aaronguo,3,2,0
1,adrianwei,3,0,0
2,aeroanish,2,0,0
3,albertguo,3,1,0
4,alexanderfung,2,0,0
...,...,...,...,...
66,vivianwuc,1,0,0
67,wgiorza,3,0,0
68,williamding,3,0,0
69,ayushi,0,1,0


In [9]:
attendance.to_csv(os.path.join(os.getcwd(), "attendance.csv"))